In [58]:
!pip install pandas
!pip install openpyxl
!pip install yfiles_jupyter_graphs
!pip install networkx


In [59]:
import pandas as pd
import os
from yfiles_jupyter_graphs import GraphWidget


In [60]:
notebook_path = os.path.abspath("notebook.ipynb")
file_path = os.path.join(os.path.dirname(notebook_path), "source_data/berber_1.xlsx")

df = pd.read_excel(file_path)

In [61]:

# shows empty widget
w = GraphWidget()

w.orthogonal_layout()

In [62]:
#for (index, colname) in enumerate(df):
#  if index > 2:
#    print(index, df[colname].values[2])

tp_index = 1
timepoints = []
visits = []
for index in range(df.shape[1]):
  if index > 2:
    x = df.iloc[4, index]
    y = pd.isnull(df.iloc[4,index])
    print(x, y)
    if y:
      x = ""
    visits.append({ 'window': x })
    timepoints.append({ 'name': "tp_%s" % (tp_index)})
    tp_index += 1
print("T:", timepoints)
print("V:", visits)

nodes = [
  {'id': 0, 'properties': {'label': 'Entry'}},
  {'id': 99999, 'properties': {'label': 'Exit'}}
]
edges = []
node_index = 1
edge_index = 0
prev_node = None
for tp in timepoints:
  nodes.append({ 'id': node_index, 'properties': {'label': tp['name']} })
  if prev_node == None:
    edges.append( {id: edge_index, 'start': 0, 'end': node_index, 'properties': {'label': "entry"}})
  else:
    edges.append( {id: edge_index, 'start': prev_node, 'end': node_index, 'properties': {'label': "next"}})
  prev_node = node_index
  node_index += 1
  edge_index += 1
edges.append( {id: edge_index, 'start': prev_node, 'end': 99999, 'properties': {'label': "exit"}})
w.set_nodes(nodes)
w.set_edges(edges)
w

nan True
nan True
-2..2 Days False
-2..2 Days False
-2..2 Days False
-2..2 Days False
-2..2 Days False
-2..2 Days False
-2..2 Days False
0..5 Days False
0..2 Weeks False
T: [{'name': 'tp_1'}, {'name': 'tp_2'}, {'name': 'tp_3'}, {'name': 'tp_4'}, {'name': 'tp_5'}, {'name': 'tp_6'}, {'name': 'tp_7'}, {'name': 'tp_8'}, {'name': 'tp_9'}, {'name': 'tp_10'}, {'name': 'tp_11'}]
V: [{'window': ''}, {'window': ''}, {'window': '-2..2 Days'}, {'window': '-2..2 Days'}, {'window': '-2..2 Days'}, {'window': '-2..2 Days'}, {'window': '-2..2 Days'}, {'window': '-2..2 Days'}, {'window': '-2..2 Days'}, {'window': '0..5 Days'}, {'window': '0..2 Weeks'}]


/Users/daveih/Library/Python/3.10/lib/python/site-packages/jupyter_client/session.py:718: UserWarning: Message serialization failed with:
keys must be str, int, float, bool or None, not builtin_function_or_method
Supporting this message is deprecated in jupyter-client 7, please make sure your message is JSON-compliant
  content = self.pack(content)


GraphWidget(layout=Layout(height='500px', width='100%'))